In [1]:
import os

In [4]:
import pandas as pd

In [55]:
import random

In [150]:
names = ['Joey', 'Phoebe', 'Chandler', 'Ross', 'Monica', 'Rachel']
dfs = []
for name in names:
    df_entry = pd.DataFrame()
    df_exit = pd.DataFrame()
    # create synthetic timestamps between 8 and 9 AM / 5 and 6 PM
    df_entry['timestamp'] = [pd.Timestamp('2019-06-{:02d} 08:30:00'.format(3+i)) + pd.Timedelta(random.randint(-15,16), 'm') for i in range(5)]
    df_exit['timestamp'] = [pd.Timestamp('2019-06-{:02d} 17:30:00'.format(3+i)) + pd.Timedelta(random.randint(-15,16), 'm') for i in range(5)]
    df_time = pd.concat([df_entry, df_exit])
    df_time['name'] = name
    # calculate the timedeltas within the same day
    for day in df_time['timestamp'].map(lambda x: x.day).unique():
        df_time.loc[df_time['timestamp'].map(lambda x: x.day) == day, 'delta'] = df_time.loc[df_time['timestamp'].map(lambda x: x.day) == day, 'timestamp'].diff().shift(0).fillna(pd.Timedelta(seconds = 0))  
    # calculate overtime as a difference between the total work time and the standard 8 hours
    df_time.loc[df_time['delta'] > pd.Timedelta(0), 'overtime'] = df_time.loc[df_time['delta'] > pd.Timedelta(0), 'delta'] - pd.Timedelta(8,'h')
    df_time.loc[df_time['overtime'].isna(), 'overtime'] = pd.Timedelta(0)
    dfs.append(df_time)

friends_time = pd.concat(dfs)
friends_time = friends_time.sort_values('timestamp').reset_index()
friends_time.drop('index', axis = 1, inplace = True)
    

In [152]:
friends_time.sample(5)

,timestamp,name,delta,overtime
40,2019-06-06 08:30:00,Chandler,00:00:00,00:00:00
16,2019-06-04 08:46:00,Ross,00:00:00,00:00:00
21,2019-06-04 17:32:00,Joey,09:14:00,01:14:00
3,2019-06-03 08:39:00,Monica,00:00:00,00:00:00
2,2019-06-03 08:29:00,Rachel,00:00:00,00:00:00


In [153]:
friends_time.groupby('name').agg({'delta':'sum'})

#only some simple aggregations like sum, max and min work in this way on timedelta objects

,delta
name,
Chandler,1 days 21:48:00
Joey,1 days 21:05:00
Monica,1 days 20:44:00
Phoebe,1 days 20:39:00
Rachel,1 days 20:45:00
Ross,1 days 20:26:00


In [154]:
#calculate average work time and overtime
friends_time[friends_time['delta'] > pd.Timedelta(0)].groupby('name')['delta','overtime'].mean(numeric_only = False)

,delta,overtime
name,,
Chandler,09:09:36,01:09:36
Joey,09:01:00,01:01:00
Monica,08:56:48,00:56:48
Phoebe,08:55:48,00:55:48
Rachel,08:57:00,00:57:00
Ross,08:53:12,00:53:12
